<a href="https://colab.research.google.com/github/aman06012003/Emotion-detect/blob/main/Emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

In [ ]:
df = pd.read_csv('Emotion_final.csv')

In [ ]:
df

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
21454,Melissa stared at her friend in dism,fear
21455,Successive state elections have seen the gover...,fear
21456,Vincent was irritated but not dismay,fear
21457,Kendall-Hume turned back to face the dismayed ...,fear


In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [ ]:
ps = PorterStemmer()

def preprocess(line):
    review = re.sub('[^a-zA-Z]', ' ', line) #leave only characters from a to z
    review = review.lower() #lower the text
    review = review.split() #turn string into list of words
    #apply Stemming 
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #delete stop words like I, and ,OR   review = ' '.join(review)
    #trun list into sentences
    return " ".join(review)

In [ ]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df['Text']=df['Text'].apply(lambda x: preprocess(x))

In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df['N_emotion'] = label_encoder.fit_transform(df['Emotion'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,ngram_range=(1,3))#example: the course was long-> [the,the course,the course was,course, course was, course was long,...]

df_cv = cv.fit_transform(df['Text']).toarray()

In [ ]:
X_train, X_test, y_train, y_test =train_test_split(df_cv, df['N_emotion'], test_size=0.25, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# load the dataset
# split into input (X) and output (y) variables
# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='softmax'))
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=10, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/10
1610/1610 [==============================] - 6s 3ms/step - loss: 1.0133 - accuracy: 0.6298
Epoch 2/10
1610/1610 [==============================] - 4s 2ms/step - loss: 0.3298 - accuracy: 0.8929
Epoch 3/10
1610/1610 [==============================] - 4s 2ms/step - loss: 0.1861 - accuracy: 0.9395
Epoch 4/10
1610/1610 [==============================] - 6s 3ms/step - loss: 0.1214 - accuracy: 0.9614
Epoch 5/10
1610/1610 [==============================] - 4s 2ms/step - loss: 0.0862 - accuracy: 0.9732
Epoch 6/10
1610/1610 [==============================] - 4s 3ms/step - loss: 0.0641 - accuracy: 0.9807
Epoch 7/10
1610/1610 [==============================] - 5s 3ms/step - loss: 0.0472 - accuracy: 0.9863
Epoch 8/10
1610/1610 [==============================] - 5s 3ms/step - loss: 0.0369 - accuracy: 0.9894
Epoch 9/10
1610/1610 [==============================] - 4s 2ms/step - loss: 0.0309 - accuracy: 0.9905
Epoch 10/10
503/503 [==============================] - 1s 2ms/step - loss: 0.0209 

In [ ]:
text='I Love you'
text=preprocess(text)
array = cv.transform([text]).toarray()
pred = model.predict(array)
a=np.argmax(pred, axis=1)
label_encoder.inverse_transform(a)[0]

1/1 [==============================] - 0s 43ms/step


'love'

In [ ]:
import tensorflow as tf 

In [ ]:
tf.keras.models.save_model(model,'emotion_seq.h5')

In [ ]:
import pickle
pickle.dump(label_encoder, open('encoder.pkl', 'wb'))
pickle.dump(cv, open('CountVectorizer.pkl', 'wb'))